In [1]:
import warnings
warnings.filterwarnings('ignore') # hide warnings
import os
import pandas as pd
import geopandas as gpd
import numpy as np
from dyntapy.supply_data import road_network_from_place, relabel_graph
from dyntapy.demand_data import add_centroids, od_graph_from_matrix
from dyntapy.assignments import StaticAssignment
from dyntapy.visualization import show_network
from pyproj import Proj, transform

In [2]:
# Creating the network for Brussels. 

# fill in these parameters 
# IMPORTANT: Use the same parameter values for the buffer as was done in STA_prep_script!
ext = 0  # 0 or 2
buffer = 40
city = 'BRUSSEL'


network = road_network_from_place("Brussels", buffer_dist_close=buffer*1000)
network = relabel_graph(network)

composing
retrieved network graph for Brussels, with 9548 nodes and 17306 edges after processing


In [3]:
# string checks
HERE = os.path.dirname(os.path.realpath("__file__"))
data_path = HERE + os.path.sep +'STA_prep'

zoning_path = data_path + os.path.sep + "shapefile_data" + os.path.sep + city + "_" + str(buffer) + "_10" + os.path.sep + city + "_" + str(buffer) + "_10.shp"
od_path2 = data_path + os.path.sep + "od_matrix_data_ext_tr" + os.path.sep + city + "_ext_tr_" + str(buffer) + "_9_10.xlsx"
od_path0 = data_path + os.path.sep + "od_matrix_data" + os.path.sep + city + "_" + str(buffer) + "_9_.xlsx"

print('zoning path: ' + zoning_path)
print('od path (if ext = 2): ' + od_path2)
print('od path (if ext = 0): ' + od_path0)

zoning path: C:\Users\anton\IP2\STA_prep\shapefile_data\BRUSSEL_40_10\BRUSSEL_40_10.shp
od path (if ext = 2): C:\Users\anton\IP2\STA_prep\od_matrix_data_ext_tr\BRUSSEL_ext_tr_40_9_10.xlsx
od path (if ext = 0): C:\Users\anton\IP2\STA_prep\od_matrix_data\BRUSSEL_40_9_.xlsx


In [4]:
# Read in the zoning file
HERE = os.path.dirname(os.path.realpath("__file__"))
data_path = HERE + os.path.sep +'STA_prep'
zoning_path = data_path + os.path.sep + "shapefile_data" + os.path.sep + city + "_" + str(buffer) + "_10" + os.path.sep + city + "_" + str(buffer) + "_10.shp"
zoning = gpd.read_file(zoning_path) # Everything is now stored in a geopandas dataframe. 

# Retrieve zone number, x_centroid (LON) and y_centroid (LAT) from each zone. 
zone_numbers = zoning["ZONENUMMER"]
x_lamb = zoning["X_LAMB"]
x_lamb = x_lamb.to_numpy()
y_lamb = zoning["Y_LAMB"]
y_lamb = y_lamb.to_numpy()

# Project to correct CRS. 
inProj = Proj(init='epsg:31370')
outProj = Proj(init='epsg:4326')
x_centroids, y_centroids = transform(inProj,outProj,x_lamb,y_lamb)

# Add the centroids to the network. Relabelling the graph is required (see demo for reason why)
connector_type = 'link'
network = add_centroids(network, x_centroids,y_centroids,k=1, method=connector_type)
network = relabel_graph(network)

In [5]:
# Read in the od matrix 
if ext == 2:
    od_path = data_path + os.path.sep + "od_matrix_data_ext_tr" + os.path.sep + city + "_ext_tr_" + str(buffer) + "_9_10.xlsx"
if ext == 0:
    od_path = data_path + os.path.sep + "od_matrix_data" + os.path.sep + city + "_" + str(buffer) + "_9_.xlsx"

od = pd.read_excel(od_path)
od_array = od.to_numpy() # The OD matrix is now stored in a numpy array

# Create OD graph and plot demand
od_graph = od_graph_from_matrix(od_array,x_centroids,y_centroids) 
# demand plot is useless since there are too many OD-pairs 

In [6]:
# The network, zoning file and od matrix have been correctly loaded and linked by now. 
# This means that we can now run an assignment on the real demand! 
zone_tolling = True
zone_toll_value = 0.05
assignment = StaticAssignment(network,od_graph, zone_toll_value, zone_tolling)
result = assignment.run('dial_b')


init passed successfully
initial loading starts 
initial loading passed successfully


AssertionError: 

In [7]:
# The network, zoning file and od matrix have been correctly loaded and linked by now. 
# This means that we can now run an assignment on the real demand! 
cordon_toll_value = 20
cordon_tolls = np.zeros(network.number_of_edges())
cordon_tolls[10]
assignment = StaticAssignment(network,od_graph, cordon_toll_value)
result = assignment.run("dial_b")

init passed successfully
initial loading starts 
initial loading passed successfully


AssertionError: 